# Poisson Process

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 4)

λ = 2  # Poisson process rate
T_max = 10  # final time
dt = 0.01
time = np.arange(0, T_max, dt)

# poisson process
np.random.seed(42)
increments = np.random.poisson(λ * dt, size=len(time))
poisson_process = np.cumsum(increments)

# plot
def plot_poisson(t_index):
  plt.clf()
  plt.plot(time[:t_index], poisson_process[:t_index], drawstyle='steps-post')
  plt.xlabel("Time")
  plt.ylabel("Number of events")
  plt.title("Poisson Process Simulation")
  plt.grid(True)
  plt.xlim([0, T_max])
  plt.ylim([0, max(poisson_process) + 1])
  plt.show()

interact(plot_poisson, t_index=IntSlider(min=1, max=len(time), step=10, value=1));

interactive(children=(IntSlider(value=1, description='t_index', max=1000, min=1, step=10), Output()), _dom_cla…

# Birth-Death Process(by discrete time)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 4)

def birth_death_process(birth_rate, death_rate, t_max, dt, initial_population):
  # simulates a birth-death process
  time = np.arange(0, t_max, dt)
  state = [initial_population]

  np.random.seed(42)
  for _ in range(1, len(time)):
    n = state[-1]
    p_birth = birth_rate * dt
    p_death = death_rate * dt
    r = np.random.rand()
    if r < p_birth:
      state.append(n + 1)
    elif r < p_birth + p_death and n > 0:
      state.append(n - 1)
    else:
      state.append(n)

  def plot_state(t_index):
    plt.clf()
    plt.plot(time[:t_index], state[:t_index], drawstyle='steps-post')
    plt.xlabel("Time")
    plt.ylabel("Population")
    plt.title(f"Birth-Death Process (λ={birth_rate}, μ={death_rate}, Initial={initial_population})")
    plt.grid(True)
    plt.xlim([0, t_max])
    plt.ylim([0, max(state) + 1])
    plt.show()

  interact(plot_state, t_index=IntSlider(min=1, max=len(time), step=10, value=1))

In [ ]:
birth_death_process(birth_rate=4.0, death_rate=4.0, t_max=50, dt=0.01, initial_population=10)

interactive(children=(IntSlider(value=1, description='t_index', max=5000, min=1, step=10), Output()), _dom_cla…

In [ ]:
birth_death_process(birth_rate=2.0, death_rate=1.0, t_max=50, dt=0.01, initial_population=10)

interactive(children=(IntSlider(value=1, description='t_index', max=5000, min=1, step=10), Output()), _dom_cla…

In [ ]:
birth_death_process(birth_rate=1.0, death_rate=4.0, t_max=50, dt=0.01, initial_population=10)

interactive(children=(IntSlider(value=1, description='t_index', max=5000, min=1, step=10), Output()), _dom_cla…

# Birth-Death Process by CTMC

## Continuous-Time Markov Chain (CTMC) Simulation of a Birth-Death Process

This simulation models a **birth-death process** using the framework of a **Continuous-Time Markov Chain (CTMC)**. The system transitions between discrete population states over continuous time, with **state-dependent transition rates**.

---

### Theory

A **birth-death process** is a type of Markov process where the system can transition from a state i to:
- i + 1 : a **birth** or **arrival**
- i - 1 : a **death** or **departure**
- or remain in state i for a period of time before transitioning.

---

### Transition Rates

Let:
- <code>lambda _i: birth rate in state i</code>
- <code>mu_i : death rate in state i</code>
- <code>Rate_i = lambda_i + mu_i: total rate of leaving state i</code>

Then, the transition probabilities for the next state are:<br>
<code>P_i,i+1 = \lambda_i/(lambda_i + mu_i),<br>
P_i,i-1 = mu_i/(lambda_i + mu_i)<br></code>

At the boundary state <code>i = 0</code>, we assume:
<code>P_0,1 = 1</code><br>

This corresponds to a boundary condition ensuring the population cannot go negative.

---

### Waiting Time Between Transitions

The time spent in state <code>i</code> is modeled as an exponentially distributed random variable:

<code>T_Exponential~(lambda_i + mu_i)</code>

This is a fundamental property of continuous-time Markov chains, reflecting the **memoryless** nature of the process.

In [ ]:
def birth_death_ctmc(lambda_list, mu_list, t_max=20, initial_population=0):
  # CTMC birth-death process using state-dependent transition probabilities

  # Parameters:
  # lambda_list- list of _lambda_i (birth rates for each state)
  # mu_list- list of mu_i (death rates for each state)
  # t_max- total simulation time
  # initial_population- starting population
  assert len(lambda_list) == len(mu_list), "lambda_list and mu_list must be same length"
  max_state = len(lambda_list) - 1

  state = [initial_population]
  times = [0]

  np.random.seed(42)
  current_state = initial_population
  current_time = 0

  while current_time < t_max:
    _lambda = lambda_list[current_state]
    mu = mu_list[current_state]
    rate = _lambda + mu

    if rate == 0:
      break  # absorbing state

    wait_time = np.random.exponential(1 / rate)
    current_time += wait_time

    # next state
    if current_state == 0:
      next_state = 1
    else:
      p_up = _lambda / rate
      if np.random.rand() < p_up:
        next_state = min(current_state + 1, max_state)
      else:
        next_state = max(current_state - 1, 0)

    state.append(next_state)
    times.append(current_time)
    current_state = next_state

  # time-aligned stepwise array for visualization
  dense_time = np.linspace(0, t_max, 1000)
  aligned_state = np.zeros_like(dense_time)
  for i in range(1, len(times)):
    mask = (dense_time >= times[i-1]) & (dense_time < times[i])
    aligned_state[mask] = state[i-1]
  aligned_state[dense_time >= times[-1]] = state[-1]

  def plot_ctmc(t_index):
    plt.clf()
    plt.plot(dense_time[:t_index], aligned_state[:t_index], drawstyle='steps-post')
    plt.xlabel("Time")
    plt.ylabel("Population")
    plt.title("CTMC Birth-Death Process")
    plt.grid(True)
    plt.ylim([0, max(aligned_state)+1])
    plt.xlim([0, t_max])
    plt.show()

  interact(plot_ctmc, t_index=IntSlider(min=1, max=len(dense_time)-1, step=10, value=1))

In [ ]:
n = 10
_lambda = [1.0] * n
mu = [1.0] * n
_lambda[0] = 1.5  # lower birth rate at state 0
mu[0] = 0.0  # no death from state 0

birth_death_ctmc(lambda_list=_lambda, mu_list=mu, t_max=30, initial_population=5)

interactive(children=(IntSlider(value=1, description='t_index', max=999, min=1, step=10), Output()), _dom_clas…

In [ ]:
n = 20
_lambda = [2.0] * n
mu = [1.0] * n

birth_death_ctmc(lambda_list=_lambda, mu_list=mu, t_max=30, initial_population=5)

interactive(children=(IntSlider(value=1, description='t_index', max=999, min=1, step=10), Output()), _dom_clas…

In [ ]:
n = 20
_lambda = [1.0] * n
mu = [3.0] * n
# _lambda[0] = 1.5  # lower birth rate at state 0
# mu[0] = 0.0  # no death from state 0

birth_death_ctmc(lambda_list=_lambda, mu_list=mu, t_max=30, initial_population=15)

interactive(children=(IntSlider(value=1, description='t_index', max=999, min=1, step=10), Output()), _dom_clas…